In [3]:
##Importing the libraries
import numpy as np
import scipy
import scipy
from scipy.optimize import minimize
import functools 
from functools import partial
from scipy.integrate import odeint
from scipy.integrate import solve_ivp

In [4]:
#Conversion of scaling

MeV = 1 
GeV = 1000*MeV
fm  = 1
c   = 1
metres = (10**15)*fm  
cm = metres/100
km = metres*1000

second = 2.99*(10**8)*metres
kg = (second**2)/((1.6*(10**(-13)))*((metres)**2))
g = kg/1000
G = 6.67*10**(-11)*((metres)**3)/(kg*(second)**2)
dyne = g*cm/(second)**2
import math
dp = 10**-12
mbar = 1.66*(10**-27)*kg
pi = math.pi
g = kg/1000
Rs = ((2*G*(1.98847*(10**30)*kg))/((c**2)))
K1 =  ((4*pi*(Rs**3))/(1.98847*((10**30)*kg)*(c**2)))*(GeV/(fm)**3)

In [12]:
class Piecewise_Polytrope:
    
    def __init__(self, rhos=None, logp1=None, Gs=None):
        (self.alphas, self.consts, 
         self.as_, self.ps) = self._get_params(rhos, logp1, Gs)
    
    def _get_params(self, rhos=None, logp1=None, Gs=None):
    
        P_unit = (dyne/cm**2)
        E_unit = rho_unit = g * c**2 / cm**3
        m_B = 1.66e-24 * g   # mass they use to convert from n to rho.
        factor = P_unit / rho_unit
        
        if (rhos == None) and (Gs == None):
            # Outer crust from Table II.
            # Everything in cgs units.

            # Note: rhos[i] is the lower density for the polytrope with Ks[i] and
            # Gs[i].
            # These are for the fixed crust.
            rhos = np.array([0, 2.44034e7, 3.78358e11, 2.62780e12])
            Ks = np.array([6.80110e-9, 1.06186e-6, 5.32697e1, 3.99874e-8])
            Gs = np.array([1.58425, 1.28733, 0.62223, 1.35692])
            ps = np.array([k*rho**g for k, rho, g in zip(Ks, rhos, Gs)])
        
        else:
            G1, G2, G3 = Gs
            if rhos == None:
                rho1 = 10**14.7
                rho2 = 10**15.0
            else:
                rho1, rho2 = rhos[0], rhos[1]
            
            p1 = 10**logp1 * factor

            K1 = p1 / rho1**G1
            K2 = p1 / rho1**G2
            p2 = K2 * rho2**G2
            K3 = p2 / (rho2**G3)
                

            rhos = np.array([0, rho1, rho2])
            Gs = np.array([G1, G2, G3])
            Ks = np.array([K1, K2, K3])
            ps = np.array([0, p1, p2])
            

        # The pressure in table III is in dyne/cm^2 but table II is in cgs.  We
        # use cgs here so we must convert the pressure
        # factor = (u.dyne/u.cm**2) / (u.g*u.c**2 / u.cm**3)

        # Now compute the constants a_i from the relationship just below (4)
        as_ = [0.0]
        for i, rho in enumerate(rhos[1:]):
            K0, K = Ks[i:i+2]
            G0, G = Gs[i:i+2]
            a0 = as_[-1]
            e_rho = (1 + a0) + K0*rho**(G0-1.0)/(G0-1.0)
            a = e_rho - 1.0 - K*rho**(G-1.0)/(G-1.0)
            as_.append(a)

        as_ = np.asarray(as_)

        # Everything is now computed as in the paper.  We next convert things
        # back to our units.

        rho_unit = g * c**2 / cm**3
        K_unit = g * c**2 / cm**3 * (1.0 / rho_unit)**Gs

        rhos *= rho_unit
        Ks *= K_unit
        # This is the mass they use to convert from n to rho.
        m_B = 1.66e-24 * g
        ns = rhos / (m_B*c**2)
        #ns[0] = EoSMixin.n_min  # Set minimum density

        # Now convert these to Polytrope1 parameters instances
        alphas = Gs - 1.0
        consts = (1+as_) * m_B*c**2
        a_ = Ks/(Gs - 1.0) * (m_B*c**2)**Gs
        return alphas, consts, a_, ps
    
    def E_P_single(self, P, a, alpha, constant, d=0):
        """E(P) for single polytrope."""
        if P <= 0:
            return 0
        
        n_b = (P/a/alpha)**(1./(1. + alpha))
        if d == 0:
            return constant * n_b + P/alpha
        elif d == 1:
            dn_dP = n_b/(1.+alpha)/P
            return constant * dn_dP +1./alpha
        else:
            raise NotImplementedError("Only d=0 or d=1 suppoerted")
    
    def E_P(self, P, d=0):
        """Return the energy density as function of P.
        """
        
        condlist = [np.logical_and(P <= self.ps[idx-1], self.ps[idx] <= P)
                    for idx in range(1, len(self.ps))]
        funclist = [partial(self.E_P_single, a=a, alpha=alpha, constant=const)
                    for a, alpha, const in zip(self.as_, self.alphas, self.consts)]
#         if np.isscalar(P):
#             # Hack until numpy issue 5729 is fixed
#             # https://github.com/numpy/numpy/issues/5729
#             return np.select(condlist, funclist).ravel()[0](P, d=d)

        return np.piecewise(P, condlist, funclist, d=d)

class Read_EoS:
        
    def __init__(self, key=None, rhos=None, log_p1=None, Gs=None):
        
        self.eos_crust = Piecewise_Polytrope()
        if key is None:
            self.eos_core = Piecewise_Polytrope(rhos, log_p1, Gs)
        elif key in self._fits.keys():
            self.eos_core = Piecewise_Polytrope(log_p1=self._fits[key][0], Gs=self._fits[key][1:])
        else:
            raise NotImplementedError("Key not supported")
        
        self.P_trans = self.get_transition_pressure()
    
    def get_transition_pressure(self):
        
        def _res(P):
            return abs(self.eos_core.E_P(P)-self.eos_crust.E_P(P))
        
        res = minimize(_res, 10)
        
        if res.success==1:
            return res.x
        else:
            raise ValueError("no root found for crust core transition")
   
    def E_P(self, P, d=0):
        """Return the energy density as function of P.
        """
        ps = [0, self.P_trans, 1e5]
        condition1 = np.logical_and(P < ps[1], ps[0] <= P)
        condition2 = np.logical_and(P < ps[2], ps[1] <= P)
        condlist = [condition1,condition2]
        funclist = [self.eos_crust.E_P, self.eos_core.E_P]
        if np.isscalar(P):
#             # Hack until numpy issue 5729 is fixed
#             # https://github.com/numpy/numpy/issues/5729
          return np.select(condlist, funclist).ravel()[0](P, d=d)

        return np.piecewise(P, condlist, funclist, d=d)

        
        

class TOV_Solver():
    """
    Class to Solve the TOV equation
    """
    
    pscale  = (GeV/fm**3)
    E_unit = MeV / fm**3
    
    def __init__(self, scode='odeint', E_P=None, eos_object=None):
        """
        Arguments:
        ---------
        scode: type of solver (odeint or ivp)
               string-like
        E_P: Method that returns Energy density (GeV/fm^3) 
               for a given pressure (GeV/fm^3)
               function-like
        eos_object: defines the background equation of state
                 class object like
        """
        
        self.scode = scode
        if eos_object is None:
            self.E_P = E_P
        else:
            self.eos_object = eos_object
            
            self.E_P = lambda P: eos_object.E_P(P*self.pscale)/self.pscale
    
    
    def dF(self, P, y):
        """
        Return RHS of TOV equation
        
        Arguments
        ---------
        P: pressure
           float
        y: list with mass and radius
           list-like
        """
        
        m, r = y[0], y[1]
        e = self.E_P(P)
        denom = (e+P)*(m+K1*P*r**3)
        dy0 = -2*K1*r**3*e*(r-m)/denom
        dy1 = -2*r*(r-m)/denom
        return [dy0, dy1]
        
        
    def tsolve(self, pct=None, scode=None, rtol=1e-16, Ptol = 1e-10):
        """
        Return solution of TOV equation using central pressure
        
        Arguments
        ---------
        pct: central pressure (in GeV/fm^3)
             float-like
        solver: type of solver: ivp or ode
             string-like
        rtol: approximation of zero radius
        """
        
        if scode is None:
            scode = self.scode

        dP = -np.sqrt(rtol) * pct
        P1 = pct + dP
        # E1 = E_P_(P1)

        # Evaluate at midpoint
        P = (pct + P1) / 2
        E = self.E_P(P)
        E0 = self.E_P(pct)

        r1 = np.sqrt(6 / E0 * np.log(
            (E + 3 * pct) * (E + P1) / (E + pct) / (E + 3 * P1)) / K1)
        M1 = K1 * E * r1**3 / 3.
        
        ps = np.linspace(Ptol, P, 1000)

        if scode == 'ivp':
            sol = solve_ivp(self.dF, [P, Ptol], [M1, r1],
                            method='Radau',
                            dense_output=True)
            mrs = np.array([sol.sol(_p) for _p in ps[::-1]])
        elif scode == 'odeint':
            dF_int = lambda x, y: self.dF(y, x)
            sol = odeint(dF_int, [M1, r1], ps[::-1])
            mrs = sol
            m = sol[:,0]
            r = sol[:,1]
            M = []
            R = []
            M = (m[-1])
            R = (r[-1])*(Rs/km)
            return M , R
        else:
            raise NotImplementedError
            """
            t_ = tov.TOV(_eos, rtol=1e-6)
            M, R, k2 = t_.get_MRk(pct * pscale)
            sol = [M / u.M0, R / u.km]
            """
        
        return mrs, ps
    
    def get_Mmax(self, scode=None):
        """
        Return the maximum mass and pressure for the given EoS
        
        Arguments
        ---------
        scode: solver type
               string like
        """
        
        if scode is None:
            scode = self.scode
    
        def _M(P):
            mrs, ps = self.tsolve(P[0]/self.pscale, scode)
            MR = mrs[-1, :]
            return -MR[0]

        res = minimize(_M, [100*u.MeV/u.fm**3], method='Nelder-Mead', bounds= ((0.01*self.E_unit, 2000*self.E_unit),), tol=1e-6)
        return res.x, -res.fun, res.success     

In [13]:
EoS_ =  Read_EoS(rhos=[67.08101935617688*1.72827e12 , 187.0784100942605*1.72827e12], log_p1= 34.94467669372845, Gs = [3.85992626,2.62551522,1.52897461])

In [14]:
solve = TOV_Solver(scode = "odeint" , E_P=EoS_.E_P)
solve.tsolve(pct= 10**3 ,scode = "odeint")

(9.76118063963e-313, 2.06105473831641e-309)

In [10]:
p1 = 5.49490219e+01*1.6022e33
np.log10(p1)

34.94467669372845

In [11]:
np.log10(67.08101935617688*1.72827e12)

14.06421124483264